In [1]:
import pandas as pd
df = pd.read_csv('NS_Data\combined_trein_data_modified.csv')

<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\c'
C:\Users\bramm\AppData\Local\Temp\ipykernel_21108\3788822566.py:2: SyntaxWarning: invalid escape sequence '\c'
  df = pd.read_csv('NS_Data\combined_trein_data_modified.csv')


### Feature engineering
In this file I will be taking a deep look into the changes I will be making to some of the features, for example changing the date-time to different columns showing which month, which day and if it is a holiday. These temporal features are needed because I will be training a time based model. I will also be deciding what exactly I will be predicting.

#### Temporal features
The temporal features in my dataset are the timestamps, right now this is set in a single column, this column has a string in the format yyyy-mm-dd and yyyy-mm-dd-hh-mm-ss, when training a model seperating these values is vital since the model will then learn from different features and make it so it will find correlations between these values, which is not possible when having these values in a single big string.
<br><br>
First I need to change the columns I plan on using to date time, I can then use pandas functions to generate seperate columns for days, hours, years etc. For now I will be looking at departure time.

In [2]:
import numpy as np

df['departure datetime'] = pd.to_datetime(df['Stop:Departure time'], errors='coerce')
df['departure year'] = df['departure datetime'].dt.year
df['departure month'] = df['departure datetime'].dt.month
df['departure day'] = df['departure datetime'].dt.day
df['departure hour'] = df['departure datetime'].dt.hour
df['departure minute'] = df['departure datetime'].dt.minute
df['departure weekday'] = df['departure datetime'].dt.weekday

conditions = [
    df['departure month'].isin([12, 1, 2]),
    df['departure month'].isin([3, 4, 5]),
    df['departure month'].isin([6, 7, 8]),
    df['departure month'].isin([9, 10, 11])
]
choices = [4, 1, 2, 3]
df['season'] = np.select(conditions, choices)

df.head()

,Service:RDT-ID,Service:Date,Service:Type,Service:Company,Service:Train number,Service:Completely cancelled,Service:Partly cancelled,Service:Maximum delay,Stop:RDT-ID,Stop:Station code,...,Stop:Planned platform,Stop:Actual platform,departure datetime,departure year,departure month,departure day,departure hour,departure minute,departure weekday,season
0,738804,2019-01-01,Intercity,NS,1410,False,False,1,6220112,RTD,...,3,2,2019-01-01 02:00:00+01:00,2019.0,1.0,1.0,2.0,0.0,1.0,4
1,738804,2019-01-01,Intercity,NS,1410,False,False,0,6220116,DT,...,1,1,2019-01-01 02:12:00+01:00,2019.0,1.0,1.0,2.0,12.0,1.0,4
2,738804,2019-01-01,Intercity,NS,1410,False,False,0,6220120,GV,...,6,6,2019-01-01 02:21:00+01:00,2019.0,1.0,1.0,2.0,21.0,1.0,4
3,738804,2019-01-01,Intercity,NS,1410,False,False,0,6220124,LEDN,...,5b,5b,2019-01-01 02:45:00+01:00,2019.0,1.0,1.0,2.0,45.0,1.0,4
4,738804,2019-01-01,Intercity,NS,1410,False,False,0,6220128,SHL,...,3,3,2019-01-01 03:02:00+01:00,2019.0,1.0,1.0,3.0,2.0,1.0,4


The next feature that I will be using is the type of train that is driving a route, in doing this I can make a distinguisment between different types of train and thus create more accuracy for each type. The column Service:Type gives this, but this is currently still in a categorical text value, I will need to change this to a numeric value for a model to learn from this feature.

Train company / operator is another feature that will be important, in my data analysis I found that there are differences between the average delay of each operator, using this in my model as feature will most likely also improve accuracy.
<br> I could also use Train number to detect certain patterns in which trains are most likely to have failures or delays, for my first version I decided to skip this feature because this would add a lot of complications due to there being so many different trains that this would not be as helpful.